In [1]:
import pandas as pd

In [2]:
df = pd.read_json("job_postings_prepped.json")

In [11]:
df.tail(1)

,Data,Vector
15885,Area Director Business Development Nexion Heal...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
df.shape

(15885, 2)

# Data Processing

## Stop Words Removal

In [ ]:
from nltk.corpus import stopwords
# import nltk
# nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.append('nan')
stop_words.append('removed')
stop_words.remove('not')
stop_words.remove('no')
stop_words[:10]

In [ ]:
df['Data'] = df['Data'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

In [ ]:
df.head()

## Tokenise Words & Put into Matrix

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

In [ ]:
# Create a CountVectorizer instance
vectorizer = CountVectorizer()

# Preparing the data to fit_tranformation
listofData  = list(df["Data"].array)

# Create the Matrix
matrix = vectorizer.fit_transform(listofData)
matrix = matrix.toarray()

# Add into new column in df
df["Vector"] = [row.tolist() for row in matrix]
df

## Calculating L2 Distance

In [ ]:
def l2_distance(v1, v2):
    if (len(v1) != len(v2)):
        print("Vectors not of the same dimension")
        return -1
    sum = 0
    for i, pt in enumerate(v1):
        sum += (pt - v2[i])**2
    return sum ** 0.5

In [ ]:
l2_distance(df['Vector'][0], df['Vector'][1])

In [ ]:
l2DistanceDf = df.copy()
v1 = df['Vector'][0]
l2DistanceDf["Distance from v1"] = df['Vector'].apply(lambda x: l2_distance(v1,x))
l2DistanceDf

## Calculating Cosine Similarity

In [5]:
def cosine_sim(v1,v2):
    if (len(v1) != len(v2)):
        print("Vectors not of the same dimension")
        return -1
    dotProduct = 0
    sumOfSquaredV1 = 0
    sumOfSquaredV2 = 0
    for i, pt in enumerate(v1):
        dotProduct += pt*v2[i]
        sumOfSquaredV1 += pt*pt
        sumOfSquaredV2 += v2[i] * v2[i]
    
    cosineSimilarity = dotProduct/((sumOfSquaredV1 ** 0.5) * (sumOfSquaredV2 ** 0.5))
    return cosineSimilarity

In [ ]:
print(cosine_sim(df['Vector'][0], df['Vector'][1]))
print(cosine_sim(df['Vector'][0], df['Vector'][0]))

In [ ]:
cosSimDf = df.copy()
v1 = df['Vector'][0]
cosSimDf["Cosine Similarity with v1"] = df['Vector'].apply(lambda x: cosine_sim(v1,x))
cosSimDf

In [ ]:
cosSimDf[cosSimDf["Cosine Similarity with v1"] != 0]

In [ ]:
print(cosSimDf["Data"][0])
print(cosSimDf["Data"][119])
print(cosSimDf["Data"][len(cosSimDf['Data'])-1])

In [12]:
import time
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

sample_queries = ["resilient investment banker",
                  "2 years experience product manager",
                  "10 years risk analyst problem solver",
                  "tax analyst for big company",
                  "software engineer for google or amazon",
                  "video editor for advertisements with 5 year experience",
                  "full time senior head nurse position",
                  "after school math and science tutor",
                  "dietitian for professional atheletes",
                  "costume designer and makeup artist"]

elapse_time = []

for query in sample_queries:
    tempDf = df[['Data']].copy()
    start_time = time.time()
    
    new_record = pd.DataFrame([{'Data':query}])
    tempDf = pd.concat([tempDf, new_record], ignore_index=True)
    
    # Create a CountVectorizer instance
    vectorizer = CountVectorizer()
    # Preparing the data to fit_tranformation
    listofData  = list(tempDf["Data"].array)
    # Create the Matrix
    matrix = vectorizer.fit_transform(listofData)
    matrix = matrix.toarray()
    # Creating a Vector column column in df
    tempDf["Vector"] = [row.tolist() for row in matrix]
    
    print(tempDf['Data'][len(tempDf['Data'])-1])
    v1 = tempDf['Vector'][len(tempDf['Data'])-1]
    tempDf['Vector'] = tempDf['Vector'].apply(lambda x: cosine_sim(v1,x))
    
    end_time = time.time()
    tempDf.drop(df.tail(1).index,inplace=True)
    tempDf.to_csv("Linear Search Results//"+query.replace(" ","_")+".csv")
    
    elapse_time.append(end_time-start_time)

print(elapse_time)

resilient investment banker
2 years experience product manager
10 years risk analyst problem solver
tax analyst for big company
software engineer for google or amazon
video editor for advertisements with 5 year experience
full time senior head nurse position
after school math and science tutor
dietitian for professional atheletes
costume designer and makeup artist
[243.0468626022339, 162.32361245155334, 165.2898588180542, 160.19741201400757, 160.39379787445068, 225.03185367584229, 161.4221270084381, 166.50238347053528, 249.01232147216797, 162.443368434906]


In [ ]:
tempDf[tempDf['Vector'] > 0.5]

In [ ]:
[10.759878635406494, 10.649078607559204, 10.477964639663696, 10.413733005523682, 10.690922498703003, 11.268934488296509, 10.590667247772217, 10.398170709609985, 10.412941455841064, 10.494908809661865]
[243.0468626022339, 162.32361245155334, 165.2898588180542, 160.19741201400757, 160.39379787445068, 225.03185367584229, 161.4221270084381, 166.50238347053528, 249.01232147216797, 162.443368434906]